In [51]:
import pandas as pd
import numpy as np

#读入数据集
u1_base = pd.read_csv('ml-100k/u1.base', sep='\t', names=['uid', 'iid', 'rate', 'timestamp'])
u1_test = pd.read_csv('ml-100k/u1.test', sep='\t', names=['uid', 'iid', 'rate', 'timestamp'])

#初始化
user_num = u1_base['uid'].max()
item_num = u1_base['iid'].max()

rating_matrix = np.zeros((user_num, item_num), float)
y_ui = np.zeros((user_num, item_num), int)

#base记录转化为matrix
import math

import pandas as pd
import numpy as np
import random

#读入数据集
u1_base = pd.read_csv('ml-100k/u1.base', sep='\t', names=['uid', 'iid', 'rate', 'timestamp'])
u1_test = pd.read_csv('ml-100k/u1.test', sep='\t', names=['uid', 'iid', 'rate', 'timestamp'])

#初始化
user_num = u1_base['uid'].max()
item_num = u1_base['iid'].max()

rating_matrix = np.zeros((user_num, item_num), float)
y_ui = np.zeros((user_num, item_num), int)

I_u = {}
U_i = {}
#base记录转化为matrix
for index, row in u1_base.iterrows():
    user_id = row['uid'] - 1
    item_id = row['iid'] - 1
    rating = row['rate']
    if rating >= 4:
        rating_matrix[user_id, item_id] = rating
        y_ui[user_id, item_id] = 1
        if user_id not in I_u:
            I_u[user_id] = set()
        I_u[user_id].add(item_id)
        if item_id not in U_i:
            U_i[item_id] = set()
        U_i[item_id].add(user_id)

I_u_preferred = {}
U_te = set()
for index, row in u1_test.iterrows():
    user_id = row['uid'] - 1
    item_id = row['iid'] - 1
    rating = row['rate']
    if rating >= 4:
        if user_id not in I_u_preferred:
            I_u_preferred[user_id] = set()
        I_u_preferred[user_id].add(item_id)
        U_te.add(user_id)

import math

import pandas as pd
import numpy as np
import random

#读入数据集
u1_base = pd.read_csv('ml-100k/u1.base', sep='\t', names=['uid', 'iid', 'rate', 'timestamp'])
u1_test = pd.read_csv('ml-100k/u1.test', sep='\t', names=['uid', 'iid', 'rate', 'timestamp'])

#初始化
user_num = u1_base['uid'].max()
item_num = u1_base['iid'].max()

rating_matrix = np.zeros((user_num, item_num), float)
y_ui = np.zeros((user_num, item_num), int)

I_u = {}
U_i = {}
#base记录转化为matrix
for index, row in u1_base.iterrows():
    user_id = row['uid'] - 1
    item_id = row['iid'] - 1
    rating = row['rate']
    if rating >= 4:
        rating_matrix[user_id, item_id] = rating
        y_ui[user_id, item_id] = 1
        if user_id not in I_u:
            I_u[user_id] = set()
        I_u[user_id].add(item_id)
        if item_id not in U_i:
            U_i[item_id] = set()
        U_i[item_id].add(user_id)

I_u_preferred = {}
U_te = set()
for index, row in u1_test.iterrows():
    user_id = row['uid'] - 1
    item_id = row['iid'] - 1
    rating = row['rate']
    if rating >= 4:
        if user_id not in I_u_preferred:
            I_u_preferred[user_id] = set()
        I_u_preferred[user_id].add(item_id)
        U_te.add(user_id)

#全局平均
GlobalAverage = y_ui.sum() / user_num / item_num

In [52]:
def I_unob_u(user_id):
    return np.where(y_ui[user_id] == 0)[0]
def get_rec_items(u, R_HAT, k):
    unob_items = I_unob_u(u)
    items_pred = {i: R_HAT(u, i) for i in unob_items}
    # 根据预测评分降序排序
    sorted_items = sorted(items_pred.items(), key=lambda x: x[1], reverse=True)
    # 输出前 k 个物品及其评分
    top_k_items = sorted_items[:k]
    return [k for k, v in top_k_items]


In [53]:
def Pre_at_k(R_HAT, k):
    Pre = 0
    for u in U_te:
        rec_items = get_rec_items(u, R_HAT, k)
        pre_items = I_u_preferred[u]  #用户实际喜欢项目 
        Pre_u = len(set(rec_items).intersection(set(pre_items))) / k
        Pre += Pre_u
    return Pre / len(U_te)


def Rec_at_k(R_HAT, k):
    Rec = 0
    for u in U_te:
        rec_items = get_rec_items(u, R_HAT, k)
        pre_items = I_u_preferred[u]
        Rec_u = len(set(rec_items).intersection(set(pre_items))) / len(pre_items)
        Rec += Rec_u
    return Rec / len(U_te)


def NDCG_at_k(R_HAT, k):
    NDCG = 0
    for u in U_te:
        rec_items = get_rec_items(u, R_HAT, k)
        pre_items = I_u_preferred[u]
        DCG_u = 0
        for pos in range(k):
            if rec_items[pos] in pre_items:
                DCG_u += 1 / math.log(pos + 2)
        IDCG_u = sum(1.0 / math.log(pos + 2) for pos in range(min(len(pre_items), k)))  #Zu
        NDCG += DCG_u / IDCG_u
    return NDCG / len(U_te)


In [54]:
def Jaccard_Item(k,j):
    if k not in U_i or j not in U_i:
        return 0  # 如果 k 或 j 不在 U_i 中，返回 0
    inter = len(U_i[k].intersection(U_i[j]))
    uni = len(U_i[k].union(U_i[j]))
    if uni == 0:
        return 0
    return inter/uni

def Jaccard_User(w,u):
    if w not in I_u or u not in I_u:
        return 0  # 如果 k 或 j 不在 U_i 中，返回 0
    inter = len(I_u[w].intersection(I_u[u]))
    uni = len(I_u[w].union(I_u[u]))
    if uni == 0:
        return 0
    return inter/uni


In [55]:
#item similarity
s_kj = np.zeros((item_num, item_num), float)
for i in range(item_num):
    s_kj[i][i] = 1
    for k in range(i):
        coefficient = Jaccard_Item(i, k)
        s_kj[i][k] = coefficient
        s_kj[k][i] = coefficient
        
#user similarity
s_wu = np.zeros((user_num, user_num), float)
for i in range(user_num):
    s_wu[i][i] = 1
    for k in range(i):
        coefficient = Jaccard_User(i, k)
        s_wu[i][k] = coefficient
        s_wu[k][i] = coefficient


In [56]:
Kn = 50
#select Kn neighbor
def neighbor_k_item(i):
    neighbors = np.where(s_kj[i] != 0)[0]
    neighbors = neighbors[neighbors!=i]
    neighbors_with_coef = [(neighbor, s_kj[i][neighbor]) for neighbor in neighbors]
    # 按相似度排序（降序）
    sorted_neighbors = sorted(neighbors_with_coef, key=lambda x: x[1], reverse=True)
    # 获取前k个邻居
    top_k = sorted_neighbors[:Kn]
    return top_k

def neighbor_k_user(u):
    neighbors = np.where(s_wu[u] != 0)[0]
    neighbors = neighbors[neighbors!=u]
    neighbors_with_coef = [(neighbor, s_wu[u][neighbor]) for neighbor in neighbors]
    # 按相似度排序（降序）
    sorted_neighbors = sorted(neighbors_with_coef, key=lambda x: x[1], reverse=True)
    # 获取前k个邻居
    top_k = sorted_neighbors[:Kn]
    return top_k


In [57]:
def I_OCCF(u, i):
    r_hat = 0
    if i not in U_i:
        return GlobalAverage
    items_rated_by_u = I_u[u]
    top_k = neighbor_k_item(i)
    for neighbor, coef in top_k:
        if neighbor in items_rated_by_u:
            r_hat += coef
    return r_hat


def U_OCCF(u, i):
    r_hat = 0
    if i not in U_i:
        return GlobalAverage
    users_rated_j = U_i[i]
    top_k = neighbor_k_user(u)
    for neighbor, coef in top_k:
        if neighbor in users_rated_j:
            r_hat += coef
    return r_hat


def H_OCCF(u, i):
    return I_OCCF(u, i) * 0.5 + U_OCCF(u, i) * 0.5


In [39]:
print(f"I_OCCF Pre: {Pre_at_k(I_OCCF,5):.4f}")
print(f"I_OCCF Rec: {Rec_at_k(I_OCCF,5):.4f}")

I_OCCF Pre: 0.3627
I_OCCF Rec: 0.1099


In [58]:
print(f"U_OCCF Pre: {Pre_at_k(U_OCCF,5):.4f}")
print(f"U_OCCF Rec: {Rec_at_k(U_OCCF,5):.4f}")

U_OCCF Pre: 0.3781
U_OCCF Rec: 0.1215


In [59]:
print(f"H_OCCF Pre: {Pre_at_k(H_OCCF,5):.4f}")
print(f"H_OCCF Rec: {Rec_at_k(H_OCCF,5):.4f}")

H_OCCF Pre: 0.3996
H_OCCF Rec: 0.1326


In [60]:
GlobalAverage

0.027828810573687086